# CIoU & IoU 比較 
# 實驗 train_vali_test:0_0_1
#### PR curve

In [2]:
import sys
sys.path.append("..")

import pandas as pd
from models import Yolov4
from tensorflow import keras
from config import yolo_config
from iou import get_iou, calculate_ciou
from utils import DataGenerator, read_annotation_lines

import numpy as np
from glob import glob
from tqdm.notebook import tqdm

import seaborn as sns
from turtle import color
from matplotlib import style
import matplotlib.pylab as pl
import matplotlib.pyplot as plt

from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
from sklearn.metrics import PrecisionRecallDisplay
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay

import math
import cv2 as cv
import xml.etree.ElementTree as ET

import os
# 使用第二張 GPU 卡
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

# 載入模型

In [3]:
# yolo_config['score_threshold']=0.7
# print(yolo_config)

In [ ]:
import warnings
warnings.filterwarnings('ignore')
class_name_path = '../class_names/classes.txt'
model = Yolov4(weight_path=None, class_name_path=class_name_path, config=yolo_config)
model.load_weights(path="../model/202208/20220822_sopbox_0.weights")

# 影片

In [ ]:
# test_dir = 'test_i'
# test_img_paths = glob("../test_img/{}/*.png".format(test_dir))
# test_img_paths[0].split('\\')[1]


# '''
# output_img: image array
# detections: DataFram of box(x1,y1,x2,y2)
# '''
# pred_box_num = 0  # 有預測框的圖像數量
# for test_img_path in tqdm(test_img_paths):
#     test_img_no = test_img_path.split('\\')[1]
#     output_img, detections = model.predict(img_path=test_img_path, 
#                          random_color=False, # 框顏色
#                          plot_img=False, # 顯示出框選圖片
#                          show_text=True, # 框上面加上類別文字
#                          box_num=1,      # 顯示框的數量
#                          return_output=True)
#     if len(detections) == 1:
#         pred_box_num += 1
#     plt.imsave('../result_img/{}/test_{}'.format('test_i_big_box',test_img_no),output_img)

# print('有預測框的圖像數量=', pred_box_num)

# 每一個test 真實box數值

In [ ]:
true_box_PATH = '../test_img/txt/20220806_anno_test_lines_0.txt'       # 這邊要修改train/test 的路徑!!!!!!!!
# true_box_PATH = '../dataset/txt/20220822_anno_train_lines_0.txt'
# true_box_PATH = '../dataset/txt/20220822_anno_val_lines_0.txt'

true_bboxes = {}
with open(true_box_PATH) as f:
    for line in f.readlines():
        true_bboxes[line.split(' ')[0]] = tuple([int(i) for i in line.split(' ')[1].split(',')[:4]])

print('數量= ', len(true_bboxes))

# 將沒有壺腹乳頭補充進true_bbox dictionary

In [ ]:
img_paths = glob("../test_img/20220806_test_img_sopbox_1/*.png")
img_names = []
for img_path in img_paths:
    img_name = img_path.split('\\')[1]
    img_names.append(img_name)


# 將沒有壺腹乳頭補充進true_bbox dictionary
for img_name in img_names:
    if img_name not in true_bboxes:
        true_bboxes[img_name]='nan'

print('測試影像數量= ',len(true_bboxes))

# 每一個預測的box數值

In [ ]:
train = ['a', 'b', 'c', 'e', 'f','g', 'j']
val   = ['d', 'i']
test  = ['h', 'z']

PATH = '../test_img/20220806_test_img_sopbox_1/'
# PATH = '../dataset/20220822_train_vali_img_sopbox_0/'                        # 這邊要修改train/test 的路徑!!!!!!!!

pred_bboxes = []
for test_img_name in true_bboxes:#glob(os.path.join(PATH, '*.png')):
    test_img_path = PATH+test_img_name
    output_img, detections = model.predict(img_path=test_img_path, 
                                        random_color=False, # 框顏色
                                        plot_img=False, # 顯示出框選圖片
                                        show_text=True, # 框上面加上類別文字
                                        box_num=1,      # 顯示框的數量
                                        return_output=True)
    
    limit = 1#len(detections['score'])                            # 這邊要修改無限制 / 限制數量1個  !!!!!!!!!!!
    pred_box_num = len(detections['score'])  
    print("pred_box_num= ",pred_box_num)                    
    if pred_box_num > 0:       # 有預測出box
        for i in range(limit):
            score = detections[i:i+1]['score'].to_numpy()[0]
            print("score= ",score)
            if score>=0.5:                                        # 這邊放寬score限制，看混淆矩陣狀況 !!!!!!!!!!! 
                pred_bboxes.append([test_img_name,score,tuple(detections[i:i+1][['x1','y1','x2','y2']].to_numpy()[0])])
            else:
                pred_bboxes.append([test_img_name, score, 'nan'])
    else:
        pred_bboxes.append([test_img_name, 0, 'nan'])
        print('沒有預測出來有box(label)')

In [ ]:
print(len(pred_bboxes))
pred_bboxes[0]

# 真實、預測 框框 繪出

In [ ]:
# def draw_rectangle(img, true_bbox, pred_bbox):
#     True_bbox = true_bbox
#     Pred_bbox = pred_bbox
    
#     # 如果True_bbox(Pred_bbox)框沒有預測出來，就採用Pred_bbox(True_bbox)框
#     if True_bbox=='nan' and Pred_bbox!='nan':
#         cv.rectangle(img, (Pred_bbox[0], Pred_bbox[1]), (Pred_bbox[2], Pred_bbox[3]), (0, 0, 255), 2)  # Pred_bbox
#         return img
#     elif Pred_bbox=='nan' and True_bbox!='nan':
#         cv.rectangle(img, (True_bbox[0], True_bbox[1]), (True_bbox[2], True_bbox[3]), (255, 0, 0), 2)  # True_bbox
#         return img
#     # 如果都沒有預測出來，就回傳'nan'
#     elif True_bbox=='nan' and Pred_bbox=='nan':
#         return img
#     else:
#         cv.rectangle(img, (Pred_bbox[0], Pred_bbox[1]), (Pred_bbox[2], Pred_bbox[3]), (0, 0, 255), 2)  # Pred_bbox
#         cv.rectangle(img, (True_bbox[0], True_bbox[1]), (True_bbox[2], True_bbox[3]), (255, 0, 0), 2)  # True_bbox
#         # 印出ciou
#         ciou = calculate_ciou(True_bbox, Pred_bbox)
#         iou  = get_iou(True_bbox, Pred_bbox)
#         print('iou= ', round(iou,2), ', ciou= ', round(ciou,2))
        
#         return img

In [ ]:
# for p in pred_bboxes:
#     img_name  = p[0]
#     path      = PATH + img_name
#     img_array = cv.imread(path)
    
#     pred_bbox = p[2]
#     img       = draw_rectangle(img_array, true_bboxes[img_name], pred_bbox)

#     print("img_name= ", img_name)
#     plt.imshow(cv.cvtColor(img, cv.COLOR_BGR2RGB))
#     plt.show()
#     print('----------------------------------------------------------')

# 計算iou
參數:

iou_threshold = 0.7    "iou大於0.7才是有框選到"

In [ ]:
def valid(TP, FN, FP, TN):
    
    precision = 0 if (TP+FP)==0 else TP/(TP+FP)
    recall    = 0 if (TP+FN)==0 else TP/(TP+FN)
    f1        = 0 if (precision+recall)==0 else 2*precision*recall/(precision+recall)

    print('precision={:.2f}, recall={:.2f}, f1={:.2f}'.format(precision, recall, f1))
    print(TP,"|",FN,"\n"
          ,FP,"|",TN)

    return precision, recall, f1

In [ ]:
def plt_precision_recall_curve_1(y_true, y_scores):
    precision, recall, thresholds = precision_recall_curve(y_true, y_scores)
    average_precision             = average_precision_score(y_true, y_scores)
    # 用套件畫
    # disp = PrecisionRecallDisplay(precision=precision, recall=recall, average_precision=average_precision)
    # disp.plot()
    # plt.ylim([0.0, 1.05])
    # plt.xlim([0.0, 1.0])
    # plt.show()

    # 自己畫
    plt.plot(recall, precision, 'b-')   
    plt.ylim([0.0, 1.05])
    plt.xlim([0.0, 1.0])  
    plt.xlabel('recall')
    plt.ylabel('precision')
    plt.title('precision_recall_curve')
    plt.legend(['AP={}'.format(round(average_precision,2))])
    plt.show()

    # print('precision = ', precision)
    # print('recall = ', recall)
    # print('thresholds = ',thresholds)
    print('AP={:.2f}'.format(average_precision))

In [ ]:
def plt_heatmap(con_ma):
    '''
    con_ma:[[TP,FN],[FP,TN]]
    '''
    sns.set()
    f,ax=plt.subplots()
    sns.heatmap(con_ma, annot=True, ax=ax, fmt='d', cmap='YlGnBu', vmax=200, vmin=0) # 畫熱力圖
    ax.set_title('confusion matrix') # 標題
    ax.set_xlabel('predict')         # x軸
    ax.set_ylabel('true')            # y軸
    ax.xaxis.set_ticklabels(['Papilla', 'No Papilla'])
    ax.yaxis.set_ticklabels(['Papilla', 'No Papilla'])
    plt.show()

### 測試集結果

In [ ]:
iou_thresholds = np.arange(0.5, 0.6, 0.1)      # 固定ciou_threshold=0.5
# test_img_names = true_bbox.keys()
precisions, recalls, f1s = [], [], []
for iou_threshold in iou_thresholds:
    TP,FP,FN,TN = 0,0,0,0
    y_true   = []                # 0:無壺腹乳頭 1:有壺腹乳頭
    y_score = []                # confidence分數
    for p in pred_bboxes:        # p=['g_000026.png',0.856, (99, 0, 672, 670)]
        test_img_name  = p[0]
        score          = p[1]       
        pred_bbox      = p[2]

        if true_bboxes[test_img_name] != 'nan' and pred_bbox != 'nan':
            # ciou = get_iou(true_bbox[test_img_name], pred_bbox[test_img_name])           # 找出真實box和預測box的iou
            ciou = calculate_ciou(true_bboxes[test_img_name], pred_bbox)                     # 找出真實box和預測box的ciou
            
            if ciou >= iou_threshold:   # 如果>=iou，才是有預測到，TP會增加
                y_true.append(1)
                y_score.append(score)
                TP += 1
            else:
                y_true.append(0)
                y_score.append(score)
                FP += 1                 # 如果<iou，FP也會增加
        elif true_bboxes[test_img_name] != 'nan' and pred_bbox == 'nan':
            y_true.append(1)
            y_score.append(0)
            FN += 1
        elif true_bboxes[test_img_name] == 'nan' and pred_bbox != 'nan':
            y_true.append(0)
            y_score.append(score)
            FP += 1
        else:
            # y_true.append(0)
            # y_scores.append(0)
            TN += 1
    
    
    con_ma = [[TP, FN],[FP, 0]]
    print('TN= ',TN)
    plt_heatmap(con_ma)                               # 畫熱力圖
    plt_precision_recall_curve_1(y_true, y_score)     # 畫PR curve
    precision, recall, f1 = valid(TP, FN, FP, 0)      # P R f1 資訊
    print('ciou_threshold={}'.format(iou_threshold))
    print("------------------------------------------------------------------------------------")
    precisions.append(precision)
    recalls.append(recall)
    f1s.append(f1)

In [ ]:
def plt_precision_recall_curve_2(iou_thresholds, y_trues, y_scores):
    n      = len(y_trues)
    colors = pl.cm.jet(np.linspace(0,1,n))
    plt.ylim([0.0, 1.05])
    plt.xlim([0.0, 1.0])  
    plt.xlabel('recall')
    plt.ylabel('precision')
    plt.title('precision_recall_curve')

    
    for i in range(n):
        iou_threshold = round(iou_thresholds[i],2)
        y_true        = y_trues[i]
        y_score       = y_scores[i]
        precision, recall, thresholds = precision_recall_curve(y_true, y_score)
        average_precision             = round(average_precision_score(y_true, y_score),2)
        
        # 自己畫
        plt.plot(recall, precision, color=colors[i], label = "CIoU={} => AP={}".format(iou_threshold, average_precision))   
        
        # print('precision = ', precision)
        # print('recall = ', recall)
        # print('thresholds = ',thresholds)
        # print('AP={:.2f}'.format(average_precision))
    plt.legend(loc='upper left',bbox_to_anchor=(0.0, -0.3))
    plt.show()

In [ ]:
iou_thresholds = np.arange(0.1, 0.8, 0.1)      # 固定ciou_threshold=0.5

precisions, recalls, f1s = [], [], []
y_trues  = []
y_scores = []
for iou_threshold in iou_thresholds:
    y_true   = []                # 0:無壺腹乳頭 1:有壺腹乳頭
    y_score  = []                # confidence分數
    for p in pred_bboxes:        # p=['g_000026.png',0.856, (99, 0, 672, 670)]
        test_img_name  = p[0]
        score          = p[1]       
        pred_bbox      = p[2]

        if true_bboxes[test_img_name] != 'nan' and pred_bbox != 'nan':
            # ciou = get_iou(true_bbox[test_img_name], pred_bbox[test_img_name])           # 找出真實box和預測box的iou
            ciou = calculate_ciou(true_bboxes[test_img_name], pred_bbox)                     # 找出真實box和預測box的ciou
            
            if ciou >= iou_threshold:   # 如果>=iou，才是有預測到，TP會增加
                y_true.append(1)
                y_score.append(score)
            else:
                y_true.append(0)
                y_score.append(score)
        elif true_bboxes[test_img_name] != 'nan' and pred_bbox == 'nan':
            y_true.append(1)
            y_score.append(0)
        elif true_bboxes[test_img_name] == 'nan' and pred_bbox != 'nan':
            y_true.append(0)
            y_score.append(score)

    
    y_trues.append(y_true)
    y_scores.append(y_score)

plt_precision_recall_curve_2(iou_thresholds, y_trues, y_scores)     # 畫PR curve